# Getting Started with Bodo

In this notebook, we will cover the basics of using Bodo with Python data analytics applications. We strongly recommend starting with this notebook before using Bodo.
Let's get started!


---------------

Make sure that your notebook is attached to a cluster, and then let's run the following code to verify that your code is running on all cores in your cluster. 


In [1]:
%%px
import bodo
import warnings
warnings.filterwarnings("ignore")
print(f"Hello World from rank {bodo.get_rank()}. Total ranks={bodo.get_size()}")

Starting 8 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/8 [00:00<?, ?engine/s]

[stdout:0] Hello World from rank 0. Total ranks=8


[stdout:6] Hello World from rank 6. Total ranks=8


[stdout:4] Hello World from rank 4. Total ranks=8


[stdout:2] Hello World from rank 2. Total ranks=8


[stdout:5] Hello World from rank 5. Total ranks=8


[stdout:7] Hello World from rank 7. Total ranks=8


[stdout:1] Hello World from rank 1. Total ranks=8


[stdout:3] Hello World from rank 3. Total ranks=8


<br>
    
If you are running your code on the Community Edition Cluster (which has 2 c5.2xlarge nodes with 8 physical cores, you should see 8 lines of output with the form:

`[stdout:<n>] "Hello World from rank <n>. Total ranks=8" `
    


## A First Parallel Computation with Bodo

 Bodo can scale your analytics code to thousands of cores, providing orders of magnitude speed up depending on program characteristics. We'll walk through a simple data transformation example, which Bodo will automatically parallelize and run across all cores on the cluster your notebook is attached to. 

### Generate data
To begin, let's generate a simple dataset and write to a [Parquet](http://parquet.apache.org/) file:

In [2]:
import pandas as pd
import numpy as np

# 10m data points
df = pd.DataFrame(
    {
        "A": np.repeat(pd.date_range("2013-01-03", periods=1000), 10_000),
        "B": np.arange(10_000_000),
    }
)
# set some values to NA
df.iloc[np.arange(1000) * 3, 0] = pd.NA
# using row_group_size helps with efficient parallel read of data later
df.to_parquet("pd_example.pq", row_group_size=100000)
print(df)

                 A        B
0              NaT        0
1       2013-01-03        1
2       2013-01-03        2
3              NaT        3
4       2013-01-03        4
...            ...      ...
9999995 2015-09-29  9999995
9999996 2015-09-29  9999996
9999997 2015-09-29  9999997
9999998 2015-09-29  9999998
9999999 2015-09-29  9999999

[10000000 rows x 2 columns]


### Example Pandas Code

We then write a simple data transformation function in Pandas that reads the parquet file we just wrote into a dataframe, processes a column of datetime values in this dataframe, and creates two new columns:

In [3]:
import time
import pandas as pd

def data_transform():
    t0 = time.time()
    df = pd.read_parquet("pd_example.pq")
    df["B"] = df.apply(lambda r: "NA" if pd.isna(r.A) else "P1" if r.A.month < 5 else "P2", axis=1)
    df["C"] = df.A.dt.month
    t2 = time.time()
    print("Total time: {:.2f}".format(time.time()-t0))
    return df

data_transform()

Total time: 141.60


,A,B,C
0,NaT,NA,NaN
1,2013-01-03,P1,1.0
2,2013-01-03,P1,1.0
3,NaT,NA,NaN
4,2013-01-03,P1,1.0
...,...,...,...
9999995,2015-09-29,P2,9.0
9999996,2015-09-29,P2,9.0
9999997,2015-09-29,P2,9.0
9999998,2015-09-29,P2,9.0


Standard Python is quite slow for such data transforms since
1. The use of custom code inside apply() does not let Pandas run an optimized prebuilt C library in its backend. Therefore, the Python interpreter overheads dominate.
2. Python uses just a single CPU core and does not parallelize computation.

Bodo solves both of these problems as you'll find out below.

### Using the Bodo JIT Decorator
Bodo optimizes and parallelizes data workloads by providing [just-in-time (JIT) compilation](https://docs.bodo.ai/latest/bodo_parallelism/bodo_parallelism_basics/). To run the code with Bodo, all that we have to do is add the `bodo.jit` decorator to the function.

In [4]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import bodo
import time

@bodo.jit
def data_transform():
    t0 = time.time()
    df = pd.read_parquet("pd_example.pq")
    df["B"] = df.apply(lambda r: "NA" if pd.isna(r.A) else "P1" if r.A.month < 5 else "P2", axis=1)
    df["C"] = df.A.dt.month
    print("Total time: {:.2f}".format(time.time()-t0))
    return df

data_transform()

Total time: 0.91


,A,B,C
0,NaT,NA,<NA>
1,2013-01-03,P1,1
2,2013-01-03,P1,1
3,NaT,NA,<NA>
4,2013-01-03,P1,1
...,...,...,...
9999995,2015-09-29,P2,9
9999996,2015-09-29,P2,9
9999997,2015-09-29,P2,9
9999998,2015-09-29,P2,9


Even though the code is still running on a single core, it is more than 150x faster because Bodo compiles the function into a native binary, eliminating the interpreter overheads in apply.

Now let’s run the code on all the cores of your cluster (8 cores if you are using this on the Community Edition Cluster) using using the `%%px` [*magic*](https://ipyparallel.readthedocs.io/en/latest/tutorial/magics.html):

In [5]:
%%px
import pandas as pd
import time

@bodo.jit
def data_transform():
    t0 = time.time()
    df = pd.read_parquet("pd_example.pq")
    t1 = time.time()
    df["B"] = df.apply(lambda r: "NA" if pd.isna(r.A) else "P1" if r.A.month < 5 else "P2", axis=1)
    df["C"] = df.A.dt.month
    t2 = time.time()
    print("IO time: {:.2f}".format(t2-t1))
    print("Compute time: {:.2f}".format(time.time()-t0))
    print("Total time: {:.2f}".format(time.time()-t0))
    return df

data_transform()

%px:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:0] IO time: 0.08
Compute time: 0.43
Total time: 0.43


,A,B,C
0,NaT,NA,<NA>
1,2013-01-03,P1,1
2,2013-01-03,P1,1
3,NaT,NA,<NA>
4,2013-01-03,P1,1
...,...,...,...
1249995,2013-05-07,P2,5
1249996,2013-05-07,P2,5
1249997,2013-05-07,P2,5
1249998,2013-05-07,P2,5


,A,B,C
7500000,2015-01-23,P1,1
7500001,2015-01-23,P1,1
7500002,2015-01-23,P1,1
7500003,2015-01-23,P1,1
7500004,2015-01-23,P1,1
...,...,...,...
8749995,2015-05-27,P2,5
8749996,2015-05-27,P2,5
8749997,2015-05-27,P2,5
8749998,2015-05-27,P2,5


,A,B,C
3750000,2014-01-13,P1,1
3750001,2014-01-13,P1,1
3750002,2014-01-13,P1,1
3750003,2014-01-13,P1,1
3750004,2014-01-13,P1,1
...,...,...,...
4999995,2014-05-17,P2,5
4999996,2014-05-17,P2,5
4999997,2014-05-17,P2,5
4999998,2014-05-17,P2,5


,A,B,C
5000000,2014-05-18,P2,5
5000001,2014-05-18,P2,5
5000002,2014-05-18,P2,5
5000003,2014-05-18,P2,5
5000004,2014-05-18,P2,5
...,...,...,...
6249995,2014-09-19,P2,9
6249996,2014-09-19,P2,9
6249997,2014-09-19,P2,9
6249998,2014-09-19,P2,9


,A,B,C
1250000,2013-05-08,P2,5
1250001,2013-05-08,P2,5
1250002,2013-05-08,P2,5
1250003,2013-05-08,P2,5
1250004,2013-05-08,P2,5
...,...,...,...
2499995,2013-09-09,P2,9
2499996,2013-09-09,P2,9
2499997,2013-09-09,P2,9
2499998,2013-09-09,P2,9


,A,B,C
2500000,2013-09-10,P2,9
2500001,2013-09-10,P2,9
2500002,2013-09-10,P2,9
2500003,2013-09-10,P2,9
2500004,2013-09-10,P2,9
...,...,...,...
3749995,2014-01-12,P1,1
3749996,2014-01-12,P1,1
3749997,2014-01-12,P1,1
3749998,2014-01-12,P1,1


,A,B,C
8750000,2015-05-28,P2,5
8750001,2015-05-28,P2,5
8750002,2015-05-28,P2,5
8750003,2015-05-28,P2,5
8750004,2015-05-28,P2,5
...,...,...,...
9999995,2015-09-29,P2,9
9999996,2015-09-29,P2,9
9999997,2015-09-29,P2,9
9999998,2015-09-29,P2,9


,A,B,C
6250000,2014-09-20,P2,9
6250001,2014-09-20,P2,9
6250002,2014-09-20,P2,9
6250003,2014-09-20,P2,9
6250004,2014-09-20,P2,9
...,...,...,...
7499995,2015-01-22,P1,1
7499996,2015-01-22,P1,1
7499997,2015-01-22,P1,1
7499998,2015-01-22,P1,1


Although the program appears to be a regular sequential Python program, Bodo compiles and *transforms* the decorated code (the `data_transform` function in this example) under the hood, so that it can run in parallel on many cores across the cluster. Each core operates on a different chunk of the data and communicates with other cores when necessary. The speedup depends on the data and program characteristics, as well as the number of cores used. Usually, we can continue scaling to many more cores as long as the data is large enough.

### Compilation Time and Caching
Bodo’s JIT workflow compiles the function the first time it is called, but reuses the compiled version for subsequent calls. In the previous example, we added timers inside the function to avoid measuring compilation time. Let’s move the timers outside and call the function twice:

In [6]:
@bodo.jit
def data_transform():
    df = pd.read_parquet("pd_example.pq")
    df["B"] = df.apply(lambda r: "NA" if pd.isna(r.A) else "P1" if r.A.month < 5 else "P2", axis=1)
    df["C"] = df.A.dt.month
    df.to_parquet("bodo_output.pq")


t0 = time.time()
data_transform()
print("Total time first call: {:.2f}".format(time.time()-t0))
t0 = time.time()
data_transform()
print("Total time second call: {:.2f}".format(time.time()-t0))

Total time first call: 4.69
Total time second call: 2.08


The first call is slower due to compilation of the function, but the second call reuses the compiled version and runs faster. See our docs on [caching](https://docs.bodo.ai/latest/performance/caching/?h=caching) for more information.

### Parallel Python Processes


Bodo uses the MPI parallelism model, which runs the full program on all cores from the beginning. Essentially, identical Python processes are launched on all the cores, and Bodo divides the data and computation in JIT functions to exploit any possible parallelism.

In [7]:
%%px

def load_data_pandas():
    df = pd.read_parquet("pd_example.pq")
    print("pandas dataframe: \n", df)

@bodo.jit
def load_data_bodo():
    df = pd.read_parquet("pd_example.pq")
    print("Bodo dataframe: \n", df)

load_data_pandas()
load_data_bodo()

[stdout:0] pandas dataframe: 
                  A        B
0              NaT        0
1       2013-01-03        1
2       2013-01-03        2
3              NaT        3
4       2013-01-03        4
...            ...      ...
9999995 2015-09-29  9999995
9999996 2015-09-29  9999996
9999997 2015-09-29  9999997
9999998 2015-09-29  9999998
9999999 2015-09-29  9999999

[10000000 rows x 2 columns]
Bodo dataframe: 
                  A        B
0              NaT        0
1       2013-01-03        1
2       2013-01-03        2
3              NaT        3
4       2013-01-03        4
...            ...      ...
1249995 2013-05-07  1249995
1249996 2013-05-07  1249996
1249997 2013-05-07  1249997
1249998 2013-05-07  1249998
1249999 2013-05-07  1249999

[1250000 rows x 2 columns]


[stdout:6] pandas dataframe: 
                  A        B
0              NaT        0
1       2013-01-03        1
2       2013-01-03        2
3              NaT        3
4       2013-01-03        4
...            ...      ...
9999995 2015-09-29  9999995
9999996 2015-09-29  9999996
9999997 2015-09-29  9999997
9999998 2015-09-29  9999998
9999999 2015-09-29  9999999

[10000000 rows x 2 columns]
Bodo dataframe: 
                  A        B
7500000 2015-01-23  7500000
7500001 2015-01-23  7500001
7500002 2015-01-23  7500002
7500003 2015-01-23  7500003
7500004 2015-01-23  7500004
...            ...      ...
8749995 2015-05-27  8749995
8749996 2015-05-27  8749996
8749997 2015-05-27  8749997
8749998 2015-05-27  8749998
8749999 2015-05-27  8749999

[1250000 rows x 2 columns]


[stdout:3] pandas dataframe: 
                  A        B
0              NaT        0
1       2013-01-03        1
2       2013-01-03        2
3              NaT        3
4       2013-01-03        4
...            ...      ...
9999995 2015-09-29  9999995
9999996 2015-09-29  9999996
9999997 2015-09-29  9999997
9999998 2015-09-29  9999998
9999999 2015-09-29  9999999

[10000000 rows x 2 columns]
Bodo dataframe: 
                  A        B
3750000 2014-01-13  3750000
3750001 2014-01-13  3750001
3750002 2014-01-13  3750002
3750003 2014-01-13  3750003
3750004 2014-01-13  3750004
...            ...      ...
4999995 2014-05-17  4999995
4999996 2014-05-17  4999996
4999997 2014-05-17  4999997
4999998 2014-05-17  4999998
4999999 2014-05-17  4999999

[1250000 rows x 2 columns]


[stdout:4] pandas dataframe: 
                  A        B
0              NaT        0
1       2013-01-03        1
2       2013-01-03        2
3              NaT        3
4       2013-01-03        4
...            ...      ...
9999995 2015-09-29  9999995
9999996 2015-09-29  9999996
9999997 2015-09-29  9999997
9999998 2015-09-29  9999998
9999999 2015-09-29  9999999

[10000000 rows x 2 columns]
Bodo dataframe: 
                  A        B
5000000 2014-05-18  5000000
5000001 2014-05-18  5000001
5000002 2014-05-18  5000002
5000003 2014-05-18  5000003
5000004 2014-05-18  5000004
...            ...      ...
6249995 2014-09-19  6249995
6249996 2014-09-19  6249996
6249997 2014-09-19  6249997
6249998 2014-09-19  6249998
6249999 2014-09-19  6249999

[1250000 rows x 2 columns]


[stdout:1] pandas dataframe: 
                  A        B
0              NaT        0
1       2013-01-03        1
2       2013-01-03        2
3              NaT        3
4       2013-01-03        4
...            ...      ...
9999995 2015-09-29  9999995
9999996 2015-09-29  9999996
9999997 2015-09-29  9999997
9999998 2015-09-29  9999998
9999999 2015-09-29  9999999

[10000000 rows x 2 columns]
Bodo dataframe: 
                  A        B
1250000 2013-05-08  1250000
1250001 2013-05-08  1250001
1250002 2013-05-08  1250002
1250003 2013-05-08  1250003
1250004 2013-05-08  1250004
...            ...      ...
2499995 2013-09-09  2499995
2499996 2013-09-09  2499996
2499997 2013-09-09  2499997
2499998 2013-09-09  2499998
2499999 2013-09-09  2499999

[1250000 rows x 2 columns]


[stdout:7] pandas dataframe: 
                  A        B
0              NaT        0
1       2013-01-03        1
2       2013-01-03        2
3              NaT        3
4       2013-01-03        4
...            ...      ...
9999995 2015-09-29  9999995
9999996 2015-09-29  9999996
9999997 2015-09-29  9999997
9999998 2015-09-29  9999998
9999999 2015-09-29  9999999

[10000000 rows x 2 columns]
Bodo dataframe: 
                  A        B
8750000 2015-05-28  8750000
8750001 2015-05-28  8750001
8750002 2015-05-28  8750002
8750003 2015-05-28  8750003
8750004 2015-05-28  8750004
...            ...      ...
9999995 2015-09-29  9999995
9999996 2015-09-29  9999996
9999997 2015-09-29  9999997
9999998 2015-09-29  9999998
9999999 2015-09-29  9999999

[1250000 rows x 2 columns]


[stdout:2] pandas dataframe: 
                  A        B
0              NaT        0
1       2013-01-03        1
2       2013-01-03        2
3              NaT        3
4       2013-01-03        4
...            ...      ...
9999995 2015-09-29  9999995
9999996 2015-09-29  9999996
9999997 2015-09-29  9999997
9999998 2015-09-29  9999998
9999999 2015-09-29  9999999

[10000000 rows x 2 columns]
Bodo dataframe: 
                  A        B
2500000 2013-09-10  2500000
2500001 2013-09-10  2500001
2500002 2013-09-10  2500002
2500003 2013-09-10  2500003
2500004 2013-09-10  2500004
...            ...      ...
3749995 2014-01-12  3749995
3749996 2014-01-12  3749996
3749997 2014-01-12  3749997
3749998 2014-01-12  3749998
3749999 2014-01-12  3749999

[1250000 rows x 2 columns]


[stdout:5] pandas dataframe: 
                  A        B
0              NaT        0
1       2013-01-03        1
2       2013-01-03        2
3              NaT        3
4       2013-01-03        4
...            ...      ...
9999995 2015-09-29  9999995
9999996 2015-09-29  9999996
9999997 2015-09-29  9999997
9999998 2015-09-29  9999998
9999999 2015-09-29  9999999

[10000000 rows x 2 columns]
Bodo dataframe: 
                  A        B
6250000 2014-09-20  6250000
6250001 2014-09-20  6250001
6250002 2014-09-20  6250002
6250003 2014-09-20  6250003
6250004 2014-09-20  6250004
...            ...      ...
7499995 2015-01-22  7499995
7499996 2015-01-22  7499996
7499997 2015-01-22  7499997
7499998 2015-01-22  7499998
7499999 2015-01-22  7499999

[1250000 rows x 2 columns]




The first eight dataframes printed in the output are regular Pandas dataframes which are *replicated* on both processes and have all 10 million rows. However, the last eight dataframes printed are Bodo parallelized Pandas dataframes, with 1.25 million rows each. In this case, Bodo parallelizes `read_parquet` automatically and loads different chunks of data into different cores. Therefore, the non-JIT parts of the Python program are *replicated* across cores whereas Bodo JIT functions are *parallelized*. 



### Parallel Computation
Bodo automatically divides computation and manages communication across cores as this example demonstrates:

In [8]:
%%px

@bodo.jit
def data_groupby():
    df = pd.read_parquet("pd_example.pq")
    df2 = df.groupby("A", as_index=False).sum()
    print(df2)


![Groupby shuffle communication pattern](img/groupby_comm.svg)

This program uses groupby which requires rows with the same key to be aggregated together. Therefore, Bodo shuffles the data automatically under the hood, and the user doesn’t need to worry about parallelism challenges like communication.


***NOTE: We recommend reading the section on [Bodo's JIT requirements](https://docs.bodo.ai/latest/dev_guide/#bodo-jit-requirements) in our docs before using Bodo for your own data applications.***  

----

If you've made it this far, you have now finished your first parallel computation with Bodo, and learned a few basics of parallelism with Bodo! Please consider joining our [community slack](https://bodocommunity.slack.com/join/shared_invite/zt-qwdc8fad-6rZ8a1RmkkJ6eOX1X__knA#/shared-invite/email) to get in touch directly with our engineers and other Bodo users like yourself. 